In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys

path = '/content/drive/My Drive/AIkazu/all.txt'
text = open(path, "r").read()
#text = text.split()

chars = sorted(list(set(text)))
print('Total chars:', len(chars))
print(chars)

Total chars: 34
['\n', ' ', '#', '(', ')', ',', '-', '/', '1', '2', '3', '4', '5', '6', '7', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'M', 'a', 'b', 'd', 'g', 'i', 'm', 'o', 's', 't', 'u']


In [ ]:
start_index_set = 7011
print(text[start_index_set : start_index_set+29])

BbM7(9) A7 Am7/D Cm7(9) Faug 


In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 29
step = 1
sentences = []
next_chars = []

for i in range(0, len(text)-maxlen, step):
   sentences.append(text[i : i+maxlen])
   next_chars.append(text[i+maxlen])

#sentences = sum(sentences, [])

# テキストのベクトル化
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
   for t, char in enumerate(sentence):
     X[i, t, char_indices[char]] = 1
   y[i, char_indices[next_chars[i]]] = 1


# モデルを定義する
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
   preds = np.asarray(preds).astype('float64')
   preds = np.log(preds) / temperature
   exp_preds = np.exp(preds)
   preds = exp_preds / np.sum(exp_preds)
   probas = np.random.multinomial(1, preds, 1)
   return np.argmax(probas)

for iteration in range(1, 20):
   print()
   print('-' *50)
   print('繰り返し回数: ', iteration)
   model.fit(X, y, batch_size=128, epochs=1)

   start_index = start_index_set

   for diversity in [0.2]:
       print()
       print('-----diveristy', diversity)

       generated = ''
       sentence = text[start_index: start_index + maxlen]
       generated += sentence
       print('----- Seedを生成しました: "' + sentence + '"')
       sys.stdout.write(generated)

       for i in range(60):
           x = np.zeros((1, maxlen, len(chars)))
           for t, char in enumerate(sentence):
               x[0, t, char_indices[char]] = 1.

           preds = model.predict(x, verbose=0)[0]
           next_index = sample(preds, diversity)
           next_char = indices_char[next_index]

           generated += next_char
           sentence = sentence[1:] + next_char

           sys.stdout.write(next_char)
           sys.stdout.flush()
       print()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



--------------------------------------------------
繰り返し回数:  1
577/577 [==============================] - 57s 94ms/step - loss: 1.2574

-----diveristy 0.2
----- Seedを生成しました: "BbM7(9) A7 Am7/D Cm7(9) Faug "
BbM7(9) A7 Am7/D Cm7(9) Faug Fm7(9) Gm7(9) Fm7(9) Gm7(9) Fm7(9) Gm7(9) Gm7(9) Gm7(9) Fm7(

--------------------------------------------------
繰り返し回数:  2
577/577 [==============================] - 55s 95ms/step - loss: 0.8975

-----diveristy 0.2
----- Seedを生成しました: "BbM7(9) A7 Am7/D Cm7(9) Faug "
BbM7(9) A7 Am7/D Cm7(9) Faug BbM7 Am7 D7 Gm7(9) Am7 Am7/D D7 Bm7 E7 Am7 D7 Bm7 E7 Am7 D7 

--------------------------------------------------
繰り返し回数:  3
577/577 [==============================] - 55s 96ms/step - loss: 0.7450

-----diveristy 0.2
----- Seedを生成しました: "BbM7(9) A7 Am7/D Cm7(9) Faug "
BbM7(9) A7 Am7/D Cm7(9) Faug BbM7(9) AbM7(9) Bb7(b9) Ebm7(9) Ab7(b9) Ebm7(9) Ab7(b9) Ebm7

--------------------------------------------------
繰り返し回数:  4
577/577 [==============================] - 55s 96